In [ ]:
import Coulomb
import numpy as np
import math
from decimal import Decimal


def single_H(hbar_omega, shell):
    return hbar_omega*(shell+1)

states = []
n = []
m = []
m_s = []
non_zero_combinations = []
NParticles=6
max_iter = 100
epsilon = 10**(-4)
hw=1


with open("nucleispnumbers.dat", "r") as qnumfile:
    for line in qnumfile:
        nums = line.split(",")
        states.append(nums[0])
        n.append(int(nums[1]))
        m.append(int(nums[2]))
        m_s.append(int(nums[4]))

spOrbitals = len(states)
two_interaction = np.zeros([spOrbitals,spOrbitals])

with open("nucleitwobody.dat", "r") as twobodyfile:
    for line in twobodyfile:
        nums = line.split(",")
        '''Matrix two_interaction has row, column indices for the first and second interacting electron respectively'''
        '''Value at those indices is the inner product of the two spin states'''
        two_interaction[int(nums[0])][int(nums[1])] = int(nums[2])

hbar_omega = 1
CMatrix = np.eye(spOrbitals) # HF coefficients
DensityMatrix = np.zeros([spOrbitals,spOrbitals])
HFMatrix = np.zeros([spOrbitals,spOrbitals])
Coulomb.write_coulomb_file(hw,nuclei_file="nucleispnumbers.dat")
CoulombMatrix = Coulomb.read_coulomb_file(spOrbitals,coulomb_file="coulomb.dat")




In [ ]:
singleparticleH = np.zeros(spOrbitals)
hf_count = 0
maxHFiter = 10
difference = 1
epsilon = 10**-3
for i in range(spOrbitals):
    singleparticleH[i] = Decimal(single_H(hbar_omega,i))
    
with open("hf_energies.txt", "w") as hffile:
    oldenergies = np.zeros(spOrbitals)
    newenergies = np.zeros(spOrbitals)
    while hf_count < maxHFiter and difference > epsilon:
        for alpha in range(spOrbitals):
            for beta in range(spOrbitals):
                
                M_s_ab = m_s[alpha] + m_s[beta]
                M_ab = m[alpha] + m[beta]
                '''Add initial term for E_a_b'''
                if beta == alpha:   
                    HFMatrix[alpha][beta] += singleparticleH[alpha]
                    spenergies, CMatrix = np.linalg.eigh(HFMatrix)
                DensityMatrix = np.zeros([spOrbitals,spOrbitals])

                for gamma in range(spOrbitals):
                    #c = particles[gamma]

                    for delta in range(spOrbitals):
                        #d = particles[delta]
                        M_s_cd = m_s[gamma] + m_s[delta]
                        M_cd = m[gamma] + m_s[delta]
                        C_sum = 0
                        direct_exchange_term = 0.0

                        '''Test for spin and M conservation'''
                        if M_s_ab == M_s_cd and M_ab == M_cd:
                            direct = two_interaction[alpha][gamma]*two_interaction[beta][delta]
                            exchange = two_interaction[alpha][delta]*two_interaction[beta][gamma]
                            '''Coulomb'''
                            '''Direct *- Coulomb(alpha,beta,gamma,delta)'''
                            #direct *= CoulombMatrix[alpha][beta][gamma][delta]
                            direct_exchange_term = (direct - exchange)*CoulombMatrix[alpha][beta][gamma][delta]/2
                            #direct_exchange_term *= CoulombMatrix[alpha][beta][gamma][delta]


                        '''Summing C terms'''
                        for j in range(NParticles):
                            C_sum += CMatrix[j][gamma]*CMatrix[j][delta]

                        '''Update Density and HF Matrix'''
                        DensityMatrix[gamma][delta] = Decimal(C_sum)# + Decimal(direct_exchange_term)
                        HFMatrix[alpha][beta] += C_sum*direct_exchange_term
                
                newenergies = spenergies
                """ Brute force computation of difference between previous and new sp HF energies """
                sum_ =0.0
                for i in range(spOrbitals):
                    sum_ += (abs(newenergies[i]-oldenergies[i]))/spOrbitals
                difference = sum_
                oldenergies = newenergies
                


                        #print("<{},{}|{},{}> = ({}*{})*({}*{}) = {}".format(a,b,c,d,a.spin_state, c.spin_state, b.spin_state, 
                        #                                                   d.spin_state, a.spin_state.dot(c.spin_state)*b.spin_state.dot(d.spin_state)))
            #HFmatrix[alpha][beta] = Decimal(sumFockTerm
        hf_count += 1
    #hffile.write(newenergies)
print("Final Hartree Fock Matrix \n {}".format(HFMatrix))
print("Final C Matrix \n {}".format(CMatrix))
#print("Final Density Matrix \n {}".format(DensityMatrix))
print("SP energies \n {}".format(spenergies))
